## Additional odds for each match to make the betting simulation more realistic

Teams that did this project claimed that they could easily make money betting using their models. However, one of the features they used when placing their imaginary bets was player attributes. However, in most big football leagues, the line-ups are published just one hour before the match starts. From the [documentation](https://www.football-data.co.uk/notes.txt) of the odds data, however, we can see that the odds are scraped usually a day or more before the match. This means that our model has an unfair advantage: it has some information when placing its bet that would not be available in real life. 

However, the odds documentation also says that for some match, they provide closing odds which are the last odds available before a match starts. When we use these odds, it would be justified to use player attributes as well because we already know who is playing the match.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fetcher import Fetcher

fetch = Fetcher('data/database.sqlite')

Fetch matches

In [3]:
matches = fetch.get_all_matches()

As in the data exploration notebook, we will also use English Premier League season 2015/16 here. This season has closing odds from bookmaker Pinnacle in the original data but not in the data available in Kaggle. In this notebook, the closing odds are added to the data.

In [4]:
import pandas as pd

plodds = pd.read_csv("data/PL16.csv")

In [5]:
plodds.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,08/08/15,Bournemouth,Aston Villa,0,1,A,0,0,D,...,1.79,26,-0.5,1.98,1.93,1.99,1.92,1.82,3.88,4.70
1,E0,08/08/15,Chelsea,Swansea,2,2,D,2,1,H,...,1.99,27,-1.5,2.24,2.16,1.80,1.73,1.37,5.04,10.88
2,E0,08/08/15,Everton,Watford,2,2,D,0,1,A,...,1.96,26,-1.0,2.28,2.18,1.76,1.71,1.75,3.76,5.44
3,E0,08/08/15,Leicester,Sunderland,4,2,H,3,0,H,...,1.67,26,-0.5,2.00,1.95,1.96,1.90,1.79,3.74,5.10
4,E0,08/08/15,Man United,Tottenham,1,0,H,1,0,H,...,2.01,26,-1.0,2.20,2.09,1.82,1.78,1.64,4.07,6.04


Let's extract the same matches from our other dataset

In [6]:
#description of these steps in the hello_data notebook
matches_df = pd.DataFrame(matches)
english = matches_df[matches_df['league_id']==1729]
english_2016 = english[english['season']=="2015/2016"]
english_2016 = english_2016.sort_values('date').reset_index(drop=True)

In [7]:
english_2016.head()

,B365A,B365D,B365H,BSA,BSD,BSH,BWA,BWD,BWH,GBA,...,home_team_api_id,home_team_goal,id,league_id,match_api_id,possession,season,shotoff,shoton,stage
0,4.00,3.6,2.00,NaN,NaN,NaN,3.70,3.30,2.00,NaN,...,8678,0,4390,1729,1987033,<possession><value><comment>58</comment><stats...,2015/2016,<shotoff><value><stats><shotoff>1</shotoff></s...,<shoton><value><stats><blocked>1</blocked></st...,1
1,11.00,5.0,1.36,NaN,NaN,NaN,9.00,4.75,1.40,NaN,...,8455,2,4391,1729,1987034,<possession><value><comment>54</comment><stats...,2015/2016,<shotoff><value><stats><shotoff>1</shotoff></s...,<shoton><value><stats><blocked>1</blocked></st...,1
2,5.50,3.9,1.70,NaN,NaN,NaN,5.00,3.50,1.70,NaN,...,8668,2,4392,1729,1987035,<possession><value><comment>65</comment><stats...,2015/2016,<shotoff><value><stats><shotoff>1</shotoff></s...,<shoton><value><stats><blocked>1</blocked></st...,1
3,4.33,3.5,1.95,NaN,NaN,NaN,3.75,3.30,2.00,NaN,...,8197,4,4393,1729,1987036,<possession><value><comment>46</comment><stats...,2015/2016,<shotoff><value><event_incident_typefk>743</ev...,<shoton><value><stats><shoton>1</shoton></stat...,1
4,6.00,4.0,1.65,NaN,NaN,NaN,5.50,4.00,1.65,NaN,...,10260,1,4394,1729,1987037,<possession><value><comment>46</comment><stats...,2015/2016,<shotoff><value><stats><shotoff>1</shotoff></s...,<shoton><value><event_incident_typefk>876</eve...,1


Fortunately for us, the odds database has the odds in chronological order, just as we did with our original dataset. Unfortunately, lots of matches start at the same time, so just around 170 of the matches seem to be in the same position in both datasets. Therefore, we have to look deeper into the data to find the matches.

In [8]:
print((english_2016['B365A'] == plodds['B365A']).sum())
print((english_2016['B365D'] == plodds['B365D']).sum())
print((english_2016['B365H'] == plodds['B365H']).sum())

170
191
171


In [9]:
#Not the most effective piece of code but it does the job
PSA = []
PSH = []
PSD = []
for match in english_2016.iterrows():
    homeid = match[1]['home_team_api_id']
    awayid = match[1]['away_team_api_id']
    home = fetch.get_team_data(homeid)[0]['team_long_name']
    away = fetch.get_team_data(awayid)[0]['team_long_name']
    for row in plodds.iterrows():
        if row[1]['HomeTeam'] == home and row[1]['AwayTeam'] == away:
            PSA.append(row[1]['PSCA'])
            PSH.append(row[1]['PSCH'])
            PSD.append(row[1]['PSCD'])
            

In [10]:
len(PSA)

90

Seems like all the names don't quite match. Let's fix that.

In [11]:
name_mapping = {'Manchester United':'Man United', 'Manchester City':'Man City', 'Norwich City': 'Norwich', 'West Ham United': 'West Ham', 'Tottenham Hotspur': 'Tottenham', 'Swansea City': 'Swansea', 'West Bromwich Albion': 'West Brom', 'Stoke City': 'Stoke', 'Leicester City': 'Leicester', 'Newcastle United': 'Newcastle'}

In [12]:
#Attempt 2
PSA = []
PSH = []
PSD = []
B365A = []
for match in english_2016.iterrows():
    homeid = match[1]['home_team_api_id']
    awayid = match[1]['away_team_api_id']
    home = fetch.get_team_data(homeid)[0]['team_long_name']
    away = fetch.get_team_data(awayid)[0]['team_long_name']
    if home in name_mapping:
        home = name_mapping[home]
    if away in name_mapping:
        away = name_mapping[away]
    for row in plodds.iterrows():
        if row[1]['HomeTeam'] == home and row[1]['AwayTeam'] == away:
            PSA.append(row[1]['PSCA'])
            PSH.append(row[1]['PSCH'])
            PSD.append(row[1]['PSCD'])
            #for checking purposes
            B365A.append(row[1]['B365A'])
            

In [13]:
assert all(english_2016['B365A'] == B365A)

Hooray!

In [14]:
english_2016['PSCA'] = PSA
english_2016['PSCH'] = PSH
english_2016['PSCD'] = PSD

In [15]:
english_2016.head()

,B365A,B365D,B365H,BSA,BSD,BSH,BWA,BWD,BWH,GBA,...,league_id,match_api_id,possession,season,shotoff,shoton,stage,PSCA,PSCH,PSCD
0,4.00,3.6,2.00,NaN,NaN,NaN,3.70,3.30,2.00,NaN,...,1729,1987033,<possession><value><comment>58</comment><stats...,2015/2016,<shotoff><value><stats><shotoff>1</shotoff></s...,<shoton><value><stats><blocked>1</blocked></st...,1,4.70,1.82,3.88
1,11.00,5.0,1.36,NaN,NaN,NaN,9.00,4.75,1.40,NaN,...,1729,1987034,<possession><value><comment>54</comment><stats...,2015/2016,<shotoff><value><stats><shotoff>1</shotoff></s...,<shoton><value><stats><blocked>1</blocked></st...,1,10.88,1.37,5.04
2,5.50,3.9,1.70,NaN,NaN,NaN,5.00,3.50,1.70,NaN,...,1729,1987035,<possession><value><comment>65</comment><stats...,2015/2016,<shotoff><value><stats><shotoff>1</shotoff></s...,<shoton><value><stats><blocked>1</blocked></st...,1,5.44,1.75,3.76
3,4.33,3.5,1.95,NaN,NaN,NaN,3.75,3.30,2.00,NaN,...,1729,1987036,<possession><value><comment>46</comment><stats...,2015/2016,<shotoff><value><event_incident_typefk>743</ev...,<shoton><value><stats><shoton>1</shoton></stat...,1,5.10,1.79,3.74
4,6.00,4.0,1.65,NaN,NaN,NaN,5.50,4.00,1.65,NaN,...,1729,1987037,<possession><value><comment>46</comment><stats...,2015/2016,<shotoff><value><stats><shotoff>1</shotoff></s...,<shoton><value><event_incident_typefk>876</eve...,1,6.04,1.64,4.07


New odds are successfully added to the data, let's write it to a csv so we can easily load it later

In [16]:
english_2016.to_csv('data/english_2016.csv')